# 環境構築

## 概要

[LeRobot][1]は、実世界のロボットのための汎用的な機械学習フレームワーク

- オープンソース機械学習コミュニティ[Hugging Face][2]公式のプロジェクト
- 模倣学習（Imitation learning）と強化学習（Reinforcement learning）の最先端の研究が[実装][3]
- SO-101を含む[オープンソースのロボット][4]を動かすことができる
- [Discordコミュニティ][5]が活発

[1]: https://github.com/huggingface/lerobot
[2]: https://huggingface.co/
[3]: https://huggingface.co/docs/lerobot/act
[4]: https://huggingface.co/docs/lerobot/so101
[5]: https://discord.com/invite/s3KuuzsPFb

## インストール

### Homebrewをインストール

```sh
/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"
```

### Miniforgeをインストール

[Miniforge][1]は、Pythonの実行環境を軽量に構築するためのツール

```sh
brew install wget

wget "https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-$(uname)-$(uname -m).sh"

bash Miniforge3-$(uname)-$(uname -m).sh
```

[1]: https://github.com/conda-forge/miniforge

### Conda環境を構築

```sh
# Python3.10をインストール
conda create -y -n lerobot python=3.10

# 注意: このコマンドはターミナルを立ち上げる度に実行する
conda activate lerobot

# FFmpegをインストール
# 学習データの動画の処理に必要
conda install ffmpeg -c conda-forge

# ffmpeg 7.Xがインストールされているか確認
conda list ffmpeg
```

### LeRobotをインストール

```sh
# LeRobot v0.4.2をダウンロード
git clone -b v0.4.2 --depth 1 https://github.com/huggingface/lerobot.git

# LeRobotディレクトリに移動
cd lerobot

# LeRobot v0.4.2をインストール
pip install -e .
```

### 追加の依存関係をインストール

```sh
# FeeTechのドライバを含む依存関係をインストール
pip install -e ".[feetech]"
```

## モーターのセットアップ

[公式ドキュメント][1]を参考にアームのモータにIDを割り振る

[1]: https://huggingface.co/docs/lerobot/so101?assembly=Leader#configure-the-motors

### フォロワーアームのUSBポートの取得

1. フォロワーアームのマイコンをUSBケーブルでパソコンに接続する
2. `lerobot.find_port`を実行する
3. USBを外してエンターキーを押す
4. 表示されるポート（例: `/dev/tty.usbmodem5A7A0182121`）をコピー
5. ポートをFOLLOWER_PORT変数に保存 （例: `FOLLOWER_PORT="/dev/tty.usbmodem5A7A0182121"`）

### フォロワーアームのモーターのセットアップ

[動画][1]を参考にモータにIDを割り当てる

コマンドを実行し、指示に従う

```sh
lerobot-setup-motors \
    --robot.type=so101_follower \
    --robot.port=$FOLLOWER_PORT
```

完了後、全ての配線を繋ぐ

[1]: https://huggingface.co/docs/lerobot/so101?assembly=Leader#setup-motors-video

### リーダーアームのUSBポートの特定

1. リーダーアームのマイコンをUSBケーブルでパソコンに接続する
2. `lerobot.find_port`を実行する
3. USBを外してエンターキーを押す
4. 表示されるポート（例: `/dev/tty.usbmodem5A7A0182121`）をコピー
5. ポートをLEADER_PORT変数に保存 （例: `LEADER_PORT="/dev/tty.usbmodem5A7A0182121"`）

### リーダーアームのモーターのセットアップ

コマンドを実行し指示に従う

```sh
lerobot-setup-motors \
    --robot.type=so101_follower \
    --robot.port=$LEADER_PORT
```

完了後、全ての配線を繋ぐ

## アームの設置

写真を参考にアームを設置する

![](asset/setup.jpg)

![](asset/setup-follower.jpg)

![](asset/setup-leader.jpg)

## キャリブレーション

### フォロワーアームのキャリブレーション

適当なフォロワーアームのIDを作成し、変数に保存

```sh
export FOLLOWER_ID=follower_1
```

キャリブレーションコマンドを実行

```sh
lerobot-calibrate \
    --robot.type=so101_follower \
    --robot.port=$FOLLOWER_PORT \
    --robot.id=$FOLLOWER_ID
```

[動画][1]を参考にしながらキャリブレーションを行う

[1]: https://huggingface.co/docs/lerobot/so101?assembly=Leader#calibration-video

### リーダーアームのキャリブレーション

適当なリーダーアームのIDを作成し、変数に保存

```sh
export LEADER_ID=leader_1
```

キャリブレーションコマンドを実行

```sh
lerobot-calibrate \
    --teleop.type=so101_leader \
    --teleop.port=$LEADER_PORT \
    --teleop.id=$LEADER_ID
```

同様にキャリブレーションを行う

[1]: https://huggingface.co/docs/lerobot/so101?assembly=Leader#calibration-video

## テレオペレーション

[ドキュメント](https://huggingface.co/docs/lerobot/il_robots#teleoperate)を参考に動作検証する

```sh
# テレオペレーションを実行
lerobot-teleoperate \
    --robot.type=so101_follower \
    --robot.port=$FOLLOWER_PORT \
    --robot.id=$FOLLOWER_ID \
    --teleop.type=so101_leader \
    --teleop.port=$LEADER_PORT \
    --teleop.id=$LEADER_ID
```

## カメラ付きテレオペレーション

[ドキュメント][1]を参考に動作検証する

```sh
# カメラ設定
export CAMERAS="{ wrist.images: {type: opencv, index_or_path: 1, width: 640, height: 480, fps: 30}, top.images: {type: opencv, index_or_path: 0, width: 640, height: 480, fps: 30}}"

# カメラつきテレオペレーションを実行
lerobot.teleoperate \
    --robot.type=so101_follower \
    --robot.port=$FOLLOWER_PORT \
    --robot.id=$FOLLOWER_ID \
    --robot.cameras=$CAMERAS \
    --teleop.type=so101_leader \
    --teleop.port=$LEADER_PORT \
    --teleop.id=$LEADER_ID \
    --display_data=true 

```

Rerun上でカメラが正しく割り当てられていることを確認する

- wrist.images -> 1
- top.images -> 0

[1]: https://huggingface.co/docs/lerobot/il_robots#teleoperate-with-cameras

## Weight & BiasesのAPIキーを設定

1. [Weights & Biases（Wandb）][1]にユーザ登録し、ログイン
2. [APIキーを取得][2]
3. `~/.zshrc`に以下を追加
    ```bash
    export WANDB_API_KEY=your_api_key
    ```
4. `source ~/.zshrc`を実行し設定を反映

[1]: https://wandb.ai/site
[2]: https://wandb.ai/authorize

## HuggingFaceのトークンを設定

1. HuggingFaceに[ログイン][1]もしくは[ユーザー登録][2]
2. [アクセストークンを作成][3]
3. `~/.zshrc`に以下を追加
    ```bash
    export HF_USER=your_huggingface_user
    export HF_TOKEN=your_huggingface_token
    ```
4. `source ~/.zshrc`を実行し設定を反映

[1]: https://huggingface.co/login
[2]: https://huggingface.co/join
[3]: https://huggingface.co/settings/tokens